<center>
    <h1 style="color:deeppink">Just between us SQL Friends...</h1>
    <h2 style="color:deeppink">RuPaul's Drag Race: Visualized</h2>
    <img src="media/header.gif" height="100px">
</center>

<h2 style="color:deeppink">Import Libraries & Connect SQLite Database</h2>

In [260]:
import pandas as pd
import sqlite3
from pandas.io.json import json_normalize

drag_race_data_base = sqlite3.connect("drag_race_data.db")

<h2 style="color:deeppink">Define Constants, Global Variables, and API endpoints</h2>

In [261]:
#API URL
DRAG_RACE_API_URL = "http://www.nokeynoshade.party/api/"

#API Endpoints - may not need all of these in the finished product
SEASONS = "seasons"
QUEENS = "queens/all"
SEASON_BY_NUMBER = "seasons/{}"

<h2 style="color:deeppink">Define Functions</h2>

In [262]:
def get_data(endpoint, SEASON = None):
    data = pd.read_json(DRAG_RACE_API_URL + endpoint.format(SEASON))
    return data

def get_seasons():
    df = pd.DataFrame()
    temp_df = pd.DataFrame()
    for i in range(len(season_id_df)):
        temp_df = get_data("seasons/{}", i + 1)
        temp_df = json_normalize(temp_df['queens'])
        temp_df['season_id'] = i + 1
        df = pd.concat([df, temp_df], sort=False)
        i += 1
    return df


<h2 style="color:deeppink">Define SQL Queries</h2>

In [280]:
queens_query = """
    SELECT season_id, queens.id, name, image_url FROM seasons
    INNER JOIN queens ON seasons.id = queens.id
    ORDER BY season_id
"""

queens_seasons_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    ORDER BY season_number, place DESC
    
"""


all_stars_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number LIKE "%A%"
    ORDER BY season_number, place DESC
"""

main_seasons_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number IN (1, 11)
    ORDER BY season_number, place DESC
    
"""


miss_congeniality_query = """
    SELECT * FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE missCongeniality IS True
    ORDER BY season_id
    
"""


<h2 style="color:deeppink">Define values used for cleanup</h2>

In [265]:
queens_drop_cols = ['queens', 'name']

<h2 style="color:deeppink">Create raw dataframes from API endpoints</h2>

In [266]:
raw_seasons_df = get_data(SEASONS)
raw_queens_df = get_data(QUEENS)

# create the season_id_df to look up season numbers by id
season_id_df = pd.DataFrame({"season_id": raw_seasons_df['id'], "season_number": raw_seasons_df['seasonNumber']})

<h2 style="color:deeppink">Clean and organize the data</h2>

<h2 style="color:deeppink">Create SQL Tables</h2>

In [267]:
# queens table
raw_queens_df.to_sql('queens', drag_race_data_base, if_exists='replace')

# seasons table
seasons_df = get_seasons().set_index('id').drop(queens_drop_cols, axis=1)
seasons_df.to_sql('seasons', drag_race_data_base, if_exists = 'replace')

# season_ids table
season_id_df.to_sql('season_ids', drag_race_data_base, if_exists='replace')

<h2 style="color:deeppink">Sandbox</h2>
<b style="color:deeppink">Use the space below to experiment. Nothing below will be included in the finished product.</b>

In [285]:
RUN_THIS_QUERY = all_stars_query

pd.read_sql(RUN_THIS_QUERY, drag_race_data_base)

pandas.core.frame.DataFrame

In [284]:
# Export the test query to a .csv file
pd.read_sql(RUN_THIS_QUERY, drag_race_data_base).to_csv('output.csv', index=True)  